In [ ]:
#| default_exp ship_dimensions
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from dataclasses import dataclass

# Ship dimension classes for different models

The idea behind the current ship dimensions is to allow for combining different pre-defined ship dimension data classes that will satisfy all resistance classes in use. Each ship dimension data class is designed to fit one resistance model class, or other classes needing ship dimensions. By creating a new ship dimension class specific for a project and inheriting from the pre defined ship dimension classes, the resulting class will ensure that all required fields are filled, and there is no need to fill in the same data twice. If additional models are implemented requiring ship dimensions, the naming convention should follow the ship dimension classes defined here, and accept an inherited ship dimension class as input.

In [ ]:
#| export
@dataclass(kw_only=True)
class ShipDimensionsHollenbachSingleScrew:
    """Ship dimensions for Hollenbach single screw ship model."""
    b_beam_m: float
    lpp_length_between_perpendiculars_m: float
    los_length_over_surface_m: float
    lwl_length_water_line_m: float
    cb_block_coefficient: float
    dp_diameter_propeller_m: float
    ta_draft_aft_m: float
    tf_draft_forward_m: float
    wetted_surface_m2: float = None
    av_transverse_area_above_water_line_m2: float = 0
    area_bilge_keel_m2: float = 0
    rho_seawater_kg_per_m3: float = 1025.0  # Seawater density
    rho_air_kg_per_m3: float = 1.225  # Air density at standard conditions (15 deg C)


@dataclass(kw_only=True)
class ShipDimensionsHollenbachTwinScrew:
    """Ship dimensions for Hollenbach twin screw ship model."""
    b_beam_m: float
    lpp_length_between_perpendiculars_m: float
    los_length_over_surface_m: float
    lwl_length_water_line_m: float
    cb_block_coefficient: float
    dp_diameter_propeller_m: float
    ta_draft_aft_m: float
    tf_draft_forward_m: float
    wetted_surface_m2: float = None
    av_transverse_area_above_water_line_m2: float = 0
    area_bilge_keel_m2: float = 0
    rho_seawater_kg_per_m3: float = 1025.0  # Seawater density
    rho_air_kg_per_m3: float = 1.225  # Air density at standard conditions (15 deg C)
    has_bulb: bool = True
    number_rudders: int = 2
    number_shaft_brackets: int = 2
    number_shaft_bossings: int = 2
    number_thrusters: int = 2


@dataclass(kw_only=True)
class ShipDimensionsAddedResistance:
    """Dimensions of a ship according to the STAWAVE2 model

    The default value for kyy_radius_gyration_in_lateral_direction_non_dim is 0.25, which is the
    recommended value from ITTC Recommended Proceduares and Guidelines 7.5-02-07-04.4

    See also: https://ittc.info/media/4180/75-02-07-044.pdf
    """
    b_beam_m: float
    lpp_length_between_perpendiculars_m: float
    cb_block_coefficient: float
    ta_draft_aft_m: float
    tf_draft_forward_m: float
    lwl_length_water_line_m: float = None
    kyy_radius_gyration_in_lateral_direction_non_dim: float = 0.25

    def __post_init__(self):
        if self.lwl_length_water_line_m is None:
            self.lwl_length_water_line_m = self.lpp_length_between_perpendiculars_m

    @property
    def tm_draft_mid_m(self) -> float:
        return 0.5 * (self.ta_draft_aft_m + self.tf_draft_forward_m)

# Test behaviour
Combining two ship dimension classes where ShipDimensionsAddedResistance expands on the ShipSimensionsHollenbachSingleScrew dataclass

In [ ]:
#| hide
from ship_model_lib.calm_water_resistance import CalmWaterResistanceHollenbachSingleScrewDesignDraft

In [ ]:
#| hide

b_beam_m=24
lpp_length_between_perpendiculars_m=145
los_length_over_surface_m=150
lwl_length_water_line_m=146.7
cb_block_coefficient=0.75
ta_draft_aft_m=8.2
tf_draft_forward_m=8.2
wetted_surface_m2=4400
av_transverse_area_above_water_line_m2=2
area_bilge_keel_m2=1
kyy_radius_gyration_in_lateral_direction_non_dim=0.26
propeller_diameter = 9.81


@dataclass
class ShipDimensionTest(ShipDimensionsHollenbachSingleScrew, ShipDimensionsAddedResistance):
    pass

test_ship_dimensions = ShipDimensionTest(b_beam_m=b_beam_m,
                                         lpp_length_between_perpendiculars_m=lpp_length_between_perpendiculars_m,
                                         los_length_over_surface_m=los_length_over_surface_m,
                                         lwl_length_water_line_m=lwl_length_water_line_m,
                                         cb_block_coefficient=cb_block_coefficient,
                                         ta_draft_aft_m=ta_draft_aft_m,
                                         tf_draft_forward_m=tf_draft_forward_m,
                                         wetted_surface_m2=wetted_surface_m2,
                                         area_bilge_keel_m2=area_bilge_keel_m2,
                                         av_transverse_area_above_water_line_m2=av_transverse_area_above_water_line_m2,
                                         kyy_radius_gyration_in_lateral_direction_non_dim=kyy_radius_gyration_in_lateral_direction_non_dim,
                                         dp_diameter_propeller_m=propeller_diameter)

calm_water_hollenbach_single_screw = CalmWaterResistanceHollenbachSingleScrewDesignDraft(ship_dimensions=test_ship_dimensions)
calm_water_hollenbach_single_screw.get_resistance_from_speed(velocity_kn=10)

150.35032255945677